In [0]:
%%capture
!pip install fastai2

In [0]:
from fastai2.vision.all import *
from shutil import copyfile

KeyboardInterrupt: ignored

In [0]:
path = Path("/content/drive/Shared drives/Asgard/Conservation Vision/Cerulean/ML/Data/")

oil_chps = np.loadtxt(path/'oil_chps.txt', dtype=str)
codes = np.loadtxt(path/'codes.txt', dtype=str)
valid_names = np.loadtxt(path/'valid_names.txt', dtype=str)

In [0]:
def get_chps(path): return [path/'chp'/(chp+'.png') for chp in oil_chps]
def get_lbls(fn): return fn.parent.parent/'lbl'/f"{fn.name}"
def splitter_func(fn): return fn.name.rsplit('_',1)[0] in valid_names # XXX Check to make sure this should be returning True for Valid

drive_files = get_chps(path)

In [0]:
lcl = Path("/content/data")
path.mkdir(exist_ok=True)
modelpath = lcl

use_lcl = False
if use_lcl:
  path = lcl
  lcl_chp = path/"chp"
  lcl_lbl = path/"lbl"
  lcl_chp.mkdir(exist_ok=True)
  lcl_lbl.mkdir(exist_ok=True)
  for i, f in enumerate(drive_files): # 10 minutes?!
    if not i%50: print(i, f)
    if not (lcl_chp/f.name).exists():
      copyfile(f, lcl_chp/f.name)
    if not (lcl_lbl/f.name).exists():
      copyfile(get_lbls(f), lcl_lbl/f.name)

In [0]:
size = 128 # Progressive resizing could happen here

In [0]:
augs = aug_transforms(flip_vert=True, max_warp=0.1, size=size)

In [0]:
seg_db = DataBlock(
    blocks=(ImageBlock, MaskBlock(codes=codes)),
    get_items = get_chps,
    splitter=FuncSplitter(splitter_func),
    batch_tfms=augs,
    get_y=get_lbls)

In [0]:
dls = SegmentationDataLoaders.from_dblock(
    dblock = seg_db,
    source = path,
    path = path,
    bs = 4*512//size)

In [0]:
dls.show_batch(vmin=0, vmax=3)

In [0]:
# Make class sparser
# add best model callback saver
# write to drive not lcl

In [0]:
arch = 50
archs = {18: resnet18, 34: resnet34, 50: resnet50}

In [0]:
learn = unet_learner(dls, archs[arch], metrics=[Dice()], model_dir=modelpath)

In [0]:
# learn.load('/content/data/34_256_.683')

In [0]:
# learn.lr_find()

In [0]:
learn.fine_tune(8, 3e-3, cbs=SaveModelCallback(monitor='dice'))

In [0]:
learn.load("model")

In [0]:
savename = f'{arch}_{size}_{round(learn.validate()[1],3)}.pkl'
learn.export(modelpath/savename)

In [0]:
learn.show_results(max_n=6, figsize=(20,20), vmin=0, vmax=3)